In [ ]:
import numpy as np
from scipy.stats import norm

print("Starting script...")

# Black-Scholes Model
def black_scholes(S, X, T, r, sigma, option_type='call'):
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        return S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return X * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

print("Black-Scholes function defined...")

# Binomial Tree Model
def binomial_tree(S, X, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize asset prices at maturity
    ST = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            ST[j, i] = S * (u ** (i - j)) * (d ** j)
    
    # Initialize option values at maturity
    option_values = np.zeros((N + 1, N + 1))
    if option_type == 'call':
        option_values[:, N] = np.maximum(0, ST[:, N] - X)
    elif option_type == 'put':
        option_values[:, N] = np.maximum(0, X - ST[:, N])
    
    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j, i] = np.exp(-r * dt) * (p * option_values[j, i + 1] + (1 - p) * option_values[j + 1, i + 1])
    
    return option_values[0, 0]

print("Binomial Tree function defined...")

# Monte Carlo Simulation
def monte_carlo_option_pricing(S, X, T, r, sigma, M, option_type='call'):
    dt = T / M
    payoff_sum = 0
    
    for i in range(M):
        ST = S
        for j in range(M):
            ST *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * np.random.normal())
        
        if option_type == 'call':
            payoff_sum += max(0, ST - X)
        elif option_type == 'put':
            payoff_sum += max(0, X - ST)
        
        # Debugging: Print intermediate stock prices and payoffs
        if i < 5:  # Print only the first few iterations to avoid clutter
            print(f"Simulation {i+1}: Final Stock Price = {ST}, Payoff = {payoff_sum / (i+1)}")
    
    option_price = np.exp(-r * T) * (payoff_sum / M)
    return option_price

print("Monte Carlo function defined...")

# Example usage
S = 100  # Current stock price
X = 100  # Strike price
T = 1    # Time to expiration in years
r = 0.05 # Risk-free interest rate
sigma = 0.2 # Volatility
N = 50   # Number of time steps for Binomial Tree
M = 10000  # Number of simulations for Monte Carlo

print("Parameters defined...")

# Black-Scholes Model
call_price_bs = black_scholes(S, X, T, r, sigma, 'call')
put_price_bs = black_scholes(S, X, T, r, sigma, 'put')
print(f"Black-Scholes Model: Call Option Price: {call_price_bs}, Put Option Price: {put_price_bs}")

# Binomial Tree Model
call_price_bt = binomial_tree(S, X, T, r, sigma, N, 'call')
put_price_bt = binomial_tree(S, X, T, r, sigma, N, 'put')
print(f"Binomial Tree Model: Call Option Price: {call_price_bt}, Put Option Price: {put_price_bt}")

# Monte Carlo Simulation
call_price_mc = monte_carlo_option_pricing(S, X, T, r, sigma, M, 'call')
put_price_mc = monte_carlo_option_pricing(S, X, T, r, sigma, M, 'put')
print(f"Monte Carlo Simulation: Call Option Price: {call_price_mc}, Put Option Price: {put_price_mc}")

print("Script completed.")


Starting script...
Black-Scholes function defined...
Binomial Tree function defined...
Monte Carlo function defined...
Parameters defined...
Black-Scholes Model: Call Option Price: 10.450583572185565, Put Option Price: 5.573526022256971
Binomial Tree Model: Call Option Price: 10.410691540732644, Put Option Price: 5.533633990803824
Simulation 1: Final Stock Price = 143.9936152033728, Payoff = 43.9936152033728
Simulation 2: Final Stock Price = 110.03115889732236, Payoff = 27.012387050347577
Simulation 3: Final Stock Price = 77.80177655234411, Payoff = 18.008258033565053
Simulation 4: Final Stock Price = 130.727180940972, Payoff = 21.18798876041679
Simulation 5: Final Stock Price = 112.58119379616419, Payoff = 19.466629767566268
